# Stable Diffusion XL Generator
Generate images using SDXL in Google Colab.

In [ ]:
!pip install --upgrade diffusers transformers accelerate safetensors gradio --quiet

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline
import gradio as gr

In [ ]:
# Load SDXL
pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")

In [ ]:
def generate(prompt):
    image = pipe(prompt=prompt).images[0]
    return image

gr.Interface(
    fn=generate,
    inputs=gr.Textbox(label="Prompt"),
    outputs=gr.Image(type="pil"),
    title="Stable Diffusion XL",
    description="Generate images from text using SDXL"
).launch()